In [95]:
from datetime import datetime,timedelta, timezone, date
import pytz

In [3]:
import html_utils
import gc_storage_utils

In [24]:
from scraper_api import ScraperAPIClient
from scrapy.http import TextResponse, HtmlResponse


In [9]:
mailjet_key = gc_storage_utils.access_secret_version(230736833082, 'mailjetKey', 1)
mailjet_secret = gc_storage_utils.access_secret_version(230736833082, 'mailjetSecret', 1)
scrapy_api_key = gc_storage_utils.access_secret_version(230736833082, 'scraper_api_key', 1)

# start scraper api client
client = ScraperAPIClient(scrapy_api_key)


# url = 'https://volunteer.covidvaccineseattle.org/'
url = 'https://communityvaccination.org'

# get html response
top_response = html_utils.request_page(url, client)

In [11]:
top_response.xpath('//div[@id="ContentPlaceHolder1_UpdatePanelOpKey"]').xpath('.//option')[1:]

[<Selector xpath='.//option' data='<option value="76">Lumen Field Site -...'>,
 <Selector xpath='.//option' data='<option value="77">Lumen Field Site -...'>,
 <Selector xpath='.//option' data='<option value="78">Lumen Field Site -...'>,
 <Selector xpath='.//option' data='<option value="82">Lumen Field Site -...'>,
 <Selector xpath='.//option' data='<option value="83">Lumen Field Site -...'>,
 <Selector xpath='.//option' data='<option value="84">Lumen Field Site -...'>]

In [12]:
view_state = top_response.xpath('//div[@class="aspNetHidden"]/input[@name="__VIEWSTATE"]/@value').get()
event_validation = top_response.xpath('//div[@class="aspNetHidden"]/input[@name="__EVENTVALIDATION"]/@value').get()

In [38]:
profession = 'G-CP2-DE'
if 'G-CP2' in profession:
    print('yes')

yes


In [14]:
profession = 'G-CP2'
prof='G'
prof_id = '175'
license_req = '0'

In [16]:
data1 = {
            'view_state':view_state, 
            'event_validation':event_validation,
            'generator':'CA0B0334',
            'profession':prof,
            'profession_ID': prof_id,
            'license_req':license_req
}

In [17]:
first_response = html_utils.make_request(data1)
# print( first_response.text)
ev1 = html_utils.get_event_viewstate(first_response)

In [18]:
data2 = {
        'view_state':ev1['view_state'], 
        'event_validation':ev1['event_validation'],
        'generator': ev1['generator'],
        'profession':prof,
        'profession_ID': prof_id,
        'license_req':license_req,
        'state_id':'47'
}
second_response = html_utils.make_request(data2)

In [19]:
ev2 = html_utils.get_event_viewstate(second_response)


In [21]:
row_id=84

In [22]:
data3 = {
        'view_state':ev2['view_state'], 
        'event_validation':ev2['event_validation'],
        'generator': ev2['generator'],
        'profession':prof,
        'profession_ID': prof_id,
        'license_req':license_req,
        'state_id':'47',
        'row_id': row_id,
}
third_response = html_utils.make_request(data3)
# print('\n\n\nthird',third_response.text)

html_text_string = html_utils.get_new_rows(third_response)

In [25]:
new_text_response = HtmlResponse(url="my HTML string", body=html_text_string, encoding='utf-8')


In [37]:
add_date = False
for tr in new_text_response.xpath('//tr'):
    for job in tr.xpath('td/div/select/option/text()').getall():
        if 'data' in job.lower():
            add_date = True


data
data
data
data


In [11]:
#first avail DATETIME '2019-02-22T00:00:00'
# retrieved timestamp'2020-12-08 18:27:52 UTC'


events= ['Wed Mar 17', 'Thu Mar 18', 'Fri Mar 19']




first_event_date = datetime.strptime(events[0], '%a %b %d').replace(year=now.year)
print(first_event_date.strftime('%Y-%m-%d'))
print(datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S.%f %Z'))

date = first_event_date.replace(tzinfo=offset)

2021-03-17
2021-03-17 19:51:48.058036 UTC


In [12]:
date

datetime.datetime(2021, 3, 17, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600)))

In [13]:
first_event_date

datetime.datetime(2021, 3, 17, 0, 0)

In [14]:
timedelta(hours=3).total_seconds() > abs((now-date).total_seconds())

False

In [15]:
timedelta(days=.5) < date-now

False

In [16]:
timedelta(hours=3).total_seconds()

10800.0

In [17]:
from google.cloud import bigquery
from google.cloud import bigquery_storage

import google.auth

In [38]:
def retreive_past_dates():
    credentials, your_project_id = google.auth.default(
        scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )
    print(your_project_id, credentials)
    # Make clients.
    bqclient = bigquery.Client(credentials=credentials, project=your_project_id,)
    bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

    query_string = """
    SELECT * 
    FROM `vaccine-volunteering.dates_accessed.dates`
    """

    dataframe = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    return dataframe

In [50]:
import pandas as pd

In [88]:
old_events = retreive_past_dates()

vaccine-volunteering <google.oauth2.credentials.Credentials object at 0x7fac4b3490a0>


In [91]:
old_events = old_events.groupby('date').max()

In [92]:
old_events.index.values

array([datetime.date(2021, 3, 17), datetime.date(2021, 3, 18),
       datetime.date(2021, 3, 19)], dtype=object)

In [93]:
def str_to_datetime(date_str):
    return datetime.strptime(date_str, '%a %b %d').date()

def date_to_bigquery(date):
    return date.strftime('%Y-%m-%d')

def timestamp_to_bigquery(date):
    return date.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S.%f %Z')

In [94]:
offset = timezone(timedelta(hours=-8))
now = datetime.now(offset)

events_dt = [str_to_datetime(date).replace(year=now.year) for date in events]

In [68]:
events_dt

[datetime.date(2021, 3, 17),
 datetime.date(2021, 3, 18),
 datetime.date(2021, 3, 19)]

In [76]:
old_events.loc[events_dt[0]].last_accessed

Timestamp('2021-03-17 20:02:21.002978+0000', tz='UTC')

In [84]:
datetime.now(pytz.UTC)

datetime.datetime(2021, 3, 17, 20, 16, 56, 266922, tzinfo=<UTC>)

In [87]:
new_events = []
for event in events_dt:
    if (event not in old_events.index.values) or \
    timedelta(hours=3).total_seconds() < abs((datetime.now(pytz.UTC)-old_events.loc[event].last_accessed).total_seconds()):
        print('time check')
        new_events.append({u"date":event, u"last_accessed":timestamp_to_bigquery(datetime.now())})
        

In [ ]:
def upload_results()

In [36]:
client = bigquery.Client()


In [37]:
errors = client.insert_rows_json('vaccine-volunteering.dates_accessed.dates', new_events)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

New rows have been added.
